## max-heap

In [17]:
# ==================
#      max-heap
# ==================

# O(log n)
def siftdown(a, i, n=None):
    if n is None:
        n = len(a)
    if i >= n:
        return
    x, c = a[i], i+i+1
    while c < n:
        if c+1 >= n:
            if x < a[c]:
                a[i], a[c] = a[c], x
            break
        l, r = a[c], a[c+1]
        if x < l and x < r:
            if r >= l:  # prefer going right to end faster
                c += 1
        elif x < r:
            c += 1
        elif x >= l:
            break
        a[i], a[c] = a[c], x
        i, c = c, c+c+1

# O(log n)
def getmax(a):
    a[0], a[-1] = a[-1], a[0]
    r = a.pop()
    siftdown(a, 0)
    return r

# O(n)
def heapify(a):
    for i in range(len(a) >> 1, -1, -1):
        siftdown(a, i)
    return a

# O(n*log n)
def heapsort(a):
    heapify(a)
    for s in range(len(a)-1, 0, -1):
        a[0], a[s] = a[s], a[0]
        siftdown(a, 0, s)
    return a

# O(log n)
def ins(a, x):
    i = len(a)
    a.append(x)
    p = i >> 1
    while i and x > a[p]:
        a[i], a[p] = a[p], x
        i, p = p, p >> 1
    return a

def test_sort(sort, niter=100, nmax=1000, rseed=123):
    from random import seed, shuffle, randint
    seed(rseed)

    assert sort([]) == []
    assert sort([1]) == [1]
    assert sort([2,2]) == [2,2]
    assert sort([4,3]) == [3,4]
    assert sort([9,5,1]) == [1,5,9]
    assert sort([9,5,1,7,3]) == [1,3,5,7,9]

    for t in range(niter):
        a = list(range(randint(1, nmax)))
        s = a[:]
        shuffle(a)
        assert sort(a) == s

    print("ok")

test_sort(sort=heapsort, niter=1000, nmax=10**3)

ok


https://stepik.org/lesson/13251/step/4

Постройте алгоритм, который по данному массиву $A[1...n]$ выводит его минимальные $\sqrt{n}$ элементов в порядке возрастания (другими словами, выводит $A′[1...\sqrt{n}]$) за время $O(n)$.

In [37]:
# finds s minumum elements of array
from math import sqrt

def smin(a, s=None):
    a = list(a)
    n = len(a)
    e = n-1
    if s is None:
        s = n
    elif str(s) == 'sqrt':
        s = int(sqrt(n))

    def down(i):
        if i >= n:
            return
        x, c = a[i], i+i+1
        while c < n:
            if c+1 >= n:
                if x > a[c]:
                    a[i], a[c] = a[c], x
                break
            l, r = a[c], a[c+1]
            if x > l and x > r:
                if r <= l:
                    c += 1
            elif x > r:
                c += 1
            elif x <= l:
                break
            a[i], a[c] = a[c], x
            i, c = c, c+c+1

    for i in range(n-1, -1, -1):
        down(i)

    o = []
    for n in range(e, e-s, -1):
        o.append(a[0])
        a[0] = a[n]
        down(0)
    return o

test_sort(sort=smin, niter=1000, nmax=10**3)

ok


In [38]:
smin([5,4,3,2,1,9,8,7,6], 'sqrt')

[1, 2, 3]

https://stepik.org/lesson/13251/step/5

Даны массивы $A[1...n]$ и $B[1...n]$. Мы хотим вывести все $n^2$ сумм вида $A[i]+B[j]$ в возрастающем порядке.
Наивный способ — создать массив, содержащий все такие суммы, и отсортировать его.
Соответствующий алгоритм имеет время работы $O(n^2\log n)$ и использует $O(n^2)$ памяти.
Приведите алгоритм с таким же временем работы, который использует линейную память.

In [35]:
# sorted sums
maxqlen = 0

def sorted_sums(a, b):
    from heapq import heappush, heappop
    global maxqlen

    ar = sorted(a)
    ac = sorted(b)
    nr, nc = len(ar), len(ac)
    # we might use only one dimension of indices
    # yet let's duplicate them for speed
    ir = [0] * nc
    ic = [0] * nr

    que = []
    heappush(que, (ar[0]+ac[0], 0, 0))
    ir[0] = ic[0] = 1

    out = []
    while que:
        # heap will have at most one row plus one column
        maxqlen = max(maxqlen, len(que))

        val, r, c = heappop(que)
        out.append(val)

        fc = c+1
        if c+1 < nc:     # try to move horizontally
            fr = ir[fc]  # should not cross a diagonal
            if fr <= r:  # enforce at most one row of elements in the queue
                heappush(que, (ar[fr]+ac[fc], fr, fc))
                ic[fr], ir[fc] = fc+1, fr+1

        fr = r+1
        if fr < nr:      # try to move vertically
            fc = ic[fr]  # should not cross a diagonal
            if fc <= c:  # enforce at most one column of elements in the queue
                heappush(que, (ar[fr]+ac[fc], fr, fc))
                ic[fr], ir[fc] = fc+1, fr+1

    return out


def test_sums(niter = 1000, nmax = 500, xmin=-10000, xmax=10000, rseed=None):
    from sys import stderr
    from random import seed, randint
    if rseed is not None:
        seed(rseed)

    # verify that maximum heap length is m+n
    global maxqlen
    maxqlen = 0

    for t in range(niter):
        a = [randint(xmin, xmax) for _ in range(randint(1, nmax))]
        b = [randint(xmin, xmax) for _ in range(randint(1, nmax))]
        sums1 = sorted_sums(a, b)
        sums2 = sorted(va+vb for va in a for vb in b)
        assert sums1 == sums2
        if t % 100 == 0:
            stderr.write(".")
    
    stderr.flush()
    print("ok maxqlen={}".format(maxqlen))

test_sums(niter=10000, nmax=100)

....................................................................................................

ok maxqlen=158
